# Analysis of Experiment 3 (Neuroscience)
In this notebook I will walk through the analysis I did for the first Experiment. There are 2 major sections:
* Preperation and First Look
* Results

## Preperation and First Look

In [1]:
#Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
from utils_openAI import *

In [3]:
#read data
df = pd.read_csv('NeuroExp3_Pilot.csv')
df.shape
df.head()

,Unnamed: 0,stimulus_number,condition,good,jargon,neuroscience,answer
0,0,0,Without Neuro Science Good,1,0,0,2
1,0,0,Neuro Science and Jargon Bad,0,1,1,-2
2,0,0,Neuro Science and Jargon Bad,0,1,1,1
3,0,1,Neuro Science and Jargon Good,1,1,1,3
4,0,1,Without Neuro Science Bad,0,0,0,-2


In [4]:
#get the value count of the different conditions
print(df['condition'].value_counts())
#get value counts of stimulous
print(df['stimulus_number'].value_counts())


Neuro Science and Jargon Bad     9
Without Neuro Science Good       7
Neuro Science and Jargon Good    6
Simple Neuro Science Bad         5
Without Neuro Science Bad        2
Simple Neuro Science Good        1
Name: condition, dtype: int64
0    14
1    12
2     4
Name: stimulus_number, dtype: int64


In [12]:
#only keep columns of interest
df = df[['stimulus_number', 'condition', 'good', 'jargon', 'neuroscience', 'answer']]
#make good, jargon and neuroscience, answer and stimulus number numeric
df['good'] = df['good'].astype(int)
df['jargon'] = df['jargon'].astype(int)
df['neuroscience'] = df['neuroscience'].astype(int)
df['answer'] = df['answer'].astype(int)
df['stimulus_number'] = df['stimulus_number'].astype(int)
#create one hot encoded version of condition
df_ohe = pd.get_dummies(df, columns = ['condition']) #One hot encode conditions in seperate df
df_ohe = pd.get_dummies(df_ohe, columns = ['stimulus_number']) #One hot encode conditions in seperate df
df_ohe.head()

,good,jargon,neuroscience,answer,condition_Neuro Science and Jargon Bad,condition_Neuro Science and Jargon Good,condition_Simple Neuro Science Bad,condition_Simple Neuro Science Good,condition_Without Neuro Science Bad,condition_Without Neuro Science Good,stimulus_number_0,stimulus_number_1,stimulus_number_2
0,1,0,0,2,0,0,0,0,0,1,1,0,0
1,0,1,1,-2,1,0,0,0,0,0,1,0,0
2,0,1,1,1,1,0,0,0,0,0,1,0,0
3,1,1,1,3,0,1,0,0,0,0,0,1,0
4,0,0,0,-2,0,0,0,0,1,0,0,1,0


## Results

In [8]:
#is rating depending on it being good, jargon, ns
#regression with answer as DV and jargon, good, neuroscience as IV
model = ols('answer ~ jargon  *  neuroscience + good', data = df_ohe).fit(cov_type = 'HC3')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 answer   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     6.559
Date:                Tue, 02 May 2023   Prob (F-statistic):            0.00189
Time:                        16:30:39   Log-Likelihood:                -55.863
No. Observations:                  30   AIC:                             119.7
Df Residuals:                      26   BIC:                             125.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.4632    

In [11]:
print(ols('answer ~ jargon', data = df_ohe).fit(cov_type = 'HC3').summary())
print(ols('answer ~ neuroscience', data = df_ohe).fit(cov_type = 'HC3').summary())
print(ols('answer ~ good', data = df_ohe).fit(cov_type = 'HC3').summary())
print(ols('answer ~ jargon  +  neuroscience + good', data = df_ohe).fit(cov_type = 'HC3').summary())

                            OLS Regression Results                            
Dep. Variable:                 answer   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.035
Method:                 Least Squares   F-statistic:                   0.02923
Date:                Tue, 02 May 2023   Prob (F-statistic):              0.865
Time:                        16:32:55   Log-Likelihood:                -64.299
No. Observations:                  30   AIC:                             132.6
Df Residuals:                      28   BIC:                             135.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8000      0.551      1.451      0.1